# **Feature Engineering**

## Key Features:
- __Mechanical Ventilation Data Analysis__
- __Organic Data Refactoring__
- __Feature Engineering__
- __Refactoring DataSets for Unsuperviced Learning Models__
---

## Maintainer

- **GitHub**: [Molderon](https://github.com/Molderon)
- **Email**: [Molderon@proton.me](mailto:Molderon@proton.me)
- **ML Model at** [ICU: Mechanical Ventilation: Asyncronies Classification](https://github.com/Molderon/ICU-Mechanical-Ventilation-Asyncronies)


> # Normal Respiratory hysteresis on **mechanical ventilation**
- Mechanical Ventilator **<->** Patient

<img src="./normalBreath2.png" alt="Scatter Matrix" style="opacity: 1;" />
<img src="./ExampleBr.png" alt="Scatter Matrix" style="opacity: 1;" />


# **Asyncronous Respiratory Hysteresis** Examples:
- Mechanical Ventilation **<->** Patient

<img src="./undefined_async1.png" alt="Scatter Matrix" style="opacity: 1;" />
<img src="./doubleTrigger3.png" alt="Scatter Matrix" style="opacity: 1;" />
<img src="./failedtrigger.png" alt="Scatter Matrix" style="opacity: 1;" />
<img src="./failed_trigger1.png" alt="Scatter Matrix" style="opacity: 1;" />

In [1]:
# --<Environment Libs>--
from dataclasses import dataclass, field
import sys, os, subprocess  
import pip, time

# --<DataScience>--
import seaborn 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.spatial import ConvexHull
from scipy.fft import fft

from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from scipy.interpolate import PchipInterpolator
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler


# --<Feature Engineering>--
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from scipy.interpolate import interp1d
from scipy.interpolate import BSpline, splrep


assert sys.version_info >= (3, 5)
bScale: bool = False
SingleSet: bool = True


<img src="./hyst_loop.jpg" alt="Hysteresis P/V Loop" style="opacity: 0.8;" />


In [2]:
#setups
plt.style.use('dark_background')
csfont = {'fontname':'Comic Sans MS'}
hfont = {'fontname':'Helvetica'}

In [3]:
def Load_LowFeq_sets():
    os.chdir("/home/molderon/Main/WorkSpace/Diploma Project/Feature Engineering/")
    Scale_Dataset = StandardScaler()

    try:
        if(SingleSet == False):
            Training_Set = pd.DataFrame()
            MultiCalss_validation = pd.DataFrame()
            CrossValidation = pd.DataFrame()

            Training_Set = pd.read_csv("TrainingSet.csv")
            MultiCalss_validation = pd.read_csv("MultiValid.csv")
            CrossValidation = pd.read_csv("CrossValid.csv")

            # Renaming bad naming concentions
            Training_Set.columns = Training_Set.columns.str.replace("u_in", "TV-i")
            Training_Set.columns = Training_Set.columns.str.replace("u_out", "TV-e")
            MultiCalss_validation.columns = MultiCalss_validation.columns.str.replace("u_in", "TV-i")
            MultiCalss_validation.columns = MultiCalss_validation.columns.str.replace("u_out", "TV-e")
            CrossValidation.columns = CrossValidation.columns.str.replace("u_in", "TV-i")
            CrossValidation.columns = CrossValidation.columns.str.replace("u_out", "TV-e")

            #Droping useless weight
            Training_Set.dropna()
            MultiCalss_validation.dropna()
            CrossValidation.dropna()

            Training_Set = Training_Set.dropna(subset=['pressure'])
            MultiCalss_validation = MultiCalss_validation.dropna(subset=['pressure'])
            CrossValidation = CrossValidation.dropna(subset=['pressure'])

            Training_Set.drop(Training_Set.columns[[0,1]], axis =1, inplace=True)
            CrossValidation.drop(CrossValidation.columns[[0,1]], axis =1, inplace=True)
            MultiCalss_validation.drop(MultiCalss_validation.columns[[0,1]], axis =1, inplace=True)

            # Optional Scaling

            if bScale == True:
                Training_Set = Scale_Dataset(Training_Set)
                MultiCalss_validation = Scale_Dataset(MultiCalss_validation)
                CrossValidation = Scale_Dataset(CrossValidation)

            print(Training_Set.describe())
            return Training_Set, MultiCalss_validation, CrossValidation

        else:
            Full_Dataset = pd.DataFrame()
            Full_Dataset = pd.read_csv("Default_MV_Data.csv")

            Full_Dataset.columns = Full_Dataset.columns.str.replace("u_in", "TV-i")
            Full_Dataset.columns = Full_Dataset.columns.str.replace("u_out", "TV-e")
            Full_Dataset.drop(Full_Dataset.columns[[0,1]], axis =1, inplace=True)
            Full_Dataset.dropna()

            if bScale == True:
                Full_Dataset = Scale_Dataset(Full_Dataset)

            return Full_Dataset
        
    except Exception:
        print(Exception.with_traceback())
        return None

In [59]:
Training_Set.iloc[:,1:].hist(bins = 50, figsize=(20,15))

array([[<Axes: title={'center': 'R'}>, <Axes: title={'center': 'C'}>],
       [<Axes: title={'center': 'time_step'}>,
        <Axes: title={'center': 'TV-i'}>],
       [<Axes: title={'center': 'TV-e'}>,
        <Axes: title={'center': 'pressure'}>]], dtype=object)

># ⚠️ **Conditional Warning: Time-Intensive Operation**
> **Condition: Datasets larger than: 300 Mbytes**

In [ ]:
def Draw_ScatterMatrix(Dataset: pd.DataFrame):    
    set_attributes: list = ["pressure", "TV-e", "TV-i","R", "C", "time_step"]
    scatter_matrix(Dataset[set_attributes], figsize=(12,8))

Main_Data = Load_LowFeq_sets()
Draw_ScatterMatrix(Main_Data)

<img src="./air_plot.png" alt="Scatter Matrix" style="opacity: 1;" />

# Scatter Matrix of organic data

<img src="./Scatter_Matrix.png" alt="Scatter Matrix" style="opacity: 1;" />

## >> |Open Questions
- Q: What is the data frequency of an individual Breath ?
- A: Frequency ​≈ 26.67 Hz
- Q: Does the timestep for individual readings remain **consistant** ?
- A: Not consistant, it's voletile between the ranges **(32 - 40 miliseconds)**

## Conclusions:
- Avg. breath cycle is 2.7 sec = **~ (80 * 7)** datapoints per breath

## >> |Feature Engineering
- Feature **A)** Reconstruction of sythetic datapoints: **Increase frequiency to ~ (32)Hz/sec.**
- Feature **B)** Create a custom **Hysteresis Cycle Metric** for each breath id.

>## Interpolation of each Dataset
- Training Set
- MultiClass_validation Set
- Corss-Validation Set

In [ ]:
def Example_Dataset(DataSet: pd.DataFrame):
   
    ExampleSet = pd.DataFrame()
    gss = GroupShuffleSplit(test_size=0.995, random_state=42) # loads 30% of Given Dataset
    for idx, _ in gss.split(DataSet, groups=DataSet['breath_id']):
        ExampleSet = DataSet.iloc[idx]
    return ExampleSet

In [10]:
# '0' == 2x the dataset Frequiency
# 'n' == 2x+n the dataset Frequency
Interloper: int = 5

In [ ]:
def Reconstruct_Cycle(DataSet: pd.DataFrame, Single_Cycle: pd.DataFrame, Interloper: int):
    # Define features and target
    X_features = Single_Cycle[['time_step', 'R', 'C', 'TV-i', 'TV-e']].values
    Y_target = Single_Cycle['pressure'].values

    # Calculate synthetic time steps with desired number of points (interpolation count)
    Interloper = min(Interloper + Single_Cycle.shape[0], 25)  # Cap at  if necessary
    Synthetic_Timestep = np.linspace(Single_Cycle['time_step'].min(), Single_Cycle['time_step'].max(), Interloper)

    # Spline for Pressure
    try:
        Symetra = splrep(Single_Cycle['time_step'], Y_target, s=10)
        Pressure_Prediction = BSpline(*Symetra)(Synthetic_Timestep)
    except Exception as e:
        print(f"Error in pressure interpolation: {e}")
        return DataSet

    # Inner function for variable reconstruction with adjustable smoothing factor
    def reconstruct_variable(target_column, smooth_factor=1):
        Y_target = Single_Cycle[target_column].values
        try:
            tck = splrep(Single_Cycle['time_step'], Y_target, s=smooth_factor)
            return BSpline(*tck)(Synthetic_Timestep)
        except Exception as e:
            print(f"Error in {target_column} interpolation: {e}")
            return np.full(Synthetic_Timestep.shape, np.nan)

    # Reconstruct each variable with consistent smoothing
    TV_i_pred = reconstruct_variable('TV-i')
    TV_e_pred = reconstruct_variable('TV-e')
    R_pred = reconstruct_variable('R')
    C_pred = reconstruct_variable('C')

    # Create DataFrame with interpolated data for the single cycle
    breath_id = Single_Cycle['breath_id'].iloc[0]
    Interpolated_Data = pd.DataFrame({
        'breath_id': np.full(Synthetic_Timestep.shape, breath_id),
        'R': R_pred,
        'C': C_pred,
        'time_step': Synthetic_Timestep,
        'TV-i': TV_i_pred,
        'TV-e': TV_e_pred,
        'pressure': Pressure_Prediction
    })

    # Append the interpolated cycle data to DataSet
    DataSet = pd.concat([DataSet, Interpolated_Data], ignore_index=True)
    
    return DataSet


In [7]:
def Reconstruct_Dataset(DataSet: pd.DataFrame, dataset_name: str, Interloper: int) -> pd.DataFrame:
    for breath_id, group in DataSet.groupby('breath_id'):
        Single_Cycle = group.sort_values(by='time_step').copy()
        
        DataSet = Reconstruct_Cycle(DataSet, Single_Cycle, Interloper)

    DataSet.to_csv(dataset_name)
    return DataSet


# Data Interpolation
 **Default Datapoints (Left) - 16 Hz/sec**   |   **Interpolation (Right) - 32 Hz/sec**

In [ ]:
def Interpolation():
      import time,warnings
      # Suppress specific Pandas warnings
      warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
      warnings.filterwarnings("ignore")
      Boosted_Dataset = pd.DataFrame()
      start = time.time()
      pd.options.mode.chained_assignment = None
      
      DataSet = Reconstruct_Dataset(Load_LowFeq_sets(),"Boosted_MV.csv", Interloper)
      
      
      ''' //Example for splitted sets
            Boosted_TrainingSet = Reconstruct_Dataset(Training_Set,"Boosted_TrainingSet.csv.csv", Interloper)
            Boosted_MultiClass =  Reconstruct_Dataset(CrossValidation,"Boosted_MultiClass.csv", Interloper)
            Boosted_CrossValidation = Reconstruct_Dataset(MultiCalss_validation,"Boosted_CrossValidation.csv", Interloper)
      '''
      
      end = time.time()
      print("Total time for interpolations is :",
            (end-start)/60, "Min")

># ⚠️ **Warning: Time-Intensive Operation**
>
> The interpolation process you're about to run is computationally expensive and may take **several hours** to complete, the author advices to use small sliced datasets.
># **Beware**
>
> - Ensure that your system has sufficient resources (CPU/RAM) to handle the process.
> - Avoid interrupting the process once it has started.
> - If possible, run this operation on a machine that will not be needed for other tasks during execution.


In [13]:
Interpolation()

Total time for interpolations is : 15.2435875137647 Min


### Interpolation Description:


1. **Input Features**: 
   $$
   X = [\text{time\_step}, R, C, TV_{i}, TV_{e}] \in \mathbb{R}^{n \times m}
   $$
   where \(n\) is the number of samples and \(m\) is the number of features.

   - \(Y\) is defined as the target variable:
   $$
   Y = \text{pressure} \in \mathbb{R}^{n}
   $$

2. **Synthetic Time Steps**: 
   - The number of interpolated points  :
   $$
   n_{\text{interloper}} = n + I
   $$
   where \(I\) is the additional points specified by the user.

   - A synthetic time step vector is created:
   $$
   t_{\text{synthetic}} = \text{linspace}(t_{\text{min}}, t_{\text{max}}, n_{\text{interloper}})
   $$

3. **Pressure Prediction Using B-Splines**: 
   - The B-spline representation of the pressure is generated using the `splrep` function:
   $$
   tck_{\text{pressure}} = \text{splrep}(S[\text{time\_step}], Y, s=0)
   $$
   - The predicted pressure values at synthetic time steps are calculated as:
   $$
   P_{\text{pred}} = BSpline(tck_{\text{pressure}})(t_{\text{synthetic}})
   $$

4. **Variable Reconstruction**:
   - A nested function is defined to reconstruct other variables in the dataset:
   $$
   V_{\text{pred}} = BSpline(tck_{\text{variable}})(t_{\text{synthetic}})
   $$

   - This is computed as:
   $$
   tck_{\text{variable}} = \text{splrep}(S[\text{time\_step}], S[\text{variable}], s=3)
   $$

5. **Constructing the Output DataFrame**: 
   - Finally, an interpolated dataset \(I\) is created:
   $$
   I = \begin{bmatrix}
   \text{breath\_id} & C_{\text{pred}} & t_{\text{synthetic}} & TV_{i_{\text{pred}}} & TV_{e_{\text{pred}}} & P_{\text{pred}} \\
   \end{bmatrix}
   $$

   - The final reconstructed dataset is obtained by concatenating the original dataset \(D\) with the interpolated data \(I\):
   $$
   D_{\text{new}} = D \cup I
   $$

In [15]:
def Plot_Hysteresis(Hysteresis_Dynamics: pd.DataFrame):
    TV_i = np.array(Hysteresis_Dynamics['TV-i'])
    TV_e = np.array(Hysteresis_Dynamics['TV-e'])
    pressure = np.array(Hysteresis_Dynamics['pressure'])

    TV_total = TV_i + TV_e

    plt.ion() 
    fig, ax = plt.subplots()
    line, = ax.plot([], [], marker='o', linestyle='-', color='b', label='Hysteresis (TV-total / Pressure)')

    ax.set_title('Hysteresis Loop: TV-total/Pressure - 35.0 Hz')
    ax.set_xlabel('TV-total (TV-i + TV-e)')
    ax.set_ylabel('Pressure')
    ax.legend()

    def update_plot(x_data, y_data):
        line.set_data(x_data, y_data)
        ax.relim()          
        ax.autoscale_view() 
        fig.canvas.draw()   
        fig.canvas.flush_events() 

    for i in range(len(TV_total)):
        update_plot(TV_total[:i+1], pressure[:i+1])

    plt.ioff()
    plt.show()

In [ ]:
def Draw_Reconstructed_Cycles(breath_id: int, DataSet: pd.DataFrame):
    Random_ids = pd.DataFrame()
    Random_ids = DataSet[DataSet['breath_id'] == breath_id].copy()
    Random_ids = Random_ids.sort_values(by="time_step")

    Hysteresis_Dynamics: dict = field(default_factory=dict)
    target_columns = ["TV-i", "TV-e","pressure", "R", "C", "time_step"]

    Hysteresis_Dynamics = Random_ids[target_columns].to_dict(orient ='list')
    Plot_Hysteresis(Hysteresis_Dynamics)

# Frequency Comparison
>  - Left(26Hz) - Default

> - Right(32Hz) - Interpolated

<img src="./17Hz.png" alt="Scatter Matrix" style="opacity: 1;" />
<img src="./32Hz.png" alt="Scatter Matrix" style="opacity: 1;" />

># Polynomial Hysteresis Area 

In [14]:
def Hysteresis_Area_Metric(DataSet: pd.DataFrame):
    os.chdir("/home/molderon/Main/WorkSpace/Diploma Project/Classification Algorithms/")
    start_time = time.time()

    if(SingleSet == False):
        Boosted_TrainingSet = Feature_Engineering(Boosted_TrainingSet)
        Boosted_TrainingSet.to_csv("Cluster_TrainingSet.csv")

        Boosted_MultiClass = Feature_Engineering(Boosted_MultiClass)
        Boosted_MultiClass.to_csv("Cluster_MultiClass.csv")

        Boosted_CrossValidation= Feature_Engineering(Boosted_CrossValidation)
        Boosted_CrossValidation.to_csv("Cluster_CrossValid.csv")
    else:
        DataSet = Feature_Engineering(DataSet)
        return DataSet

    end_time = time.time()

    print("Execution time:", (end_time - start_time)/60, "::mins")

>## Feature Engineering
- **Pressure/Volume Loop** 

# |Creating P/V cycles as a feature
- **Working with the Interloped Datasets _~(32Hz)~_**

- **The area of the polygon can be calculated using the formula:**

$$
A = \frac{1}{2} \left| \sum_{i=1}^{n} \left( x_i \cdot y_{i+1} - y_i \cdot x_{i+1} \right) \right|
$$

Where:
- \(x_i\) and \(y_i\) are the coordinates of the polygon's vertices.
- The indices wrap around, i.e., \(y_{n+1} = y_1\) and \(x_{n+1} = x_1\).
> __________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

### Area Calculation Using the Trapezoidal Rule

The area under a curve defined by discrete points can be calculated using the **trapezoidal rule**, which approximates the integral of the function. Given a set of points \((x_i, y_i)\), the area \(A\) under the curve from \(x_1\) to \(x_n\) can be represented mathematically as:

$$
A = \int_{a}^{b} f(x) \, dx \approx \sum_{i=1}^{n-1} \frac{(y_i + y_{i+1})}{2} (x_{i+1} - x_i)
$$

Where:
- \( A \) is the approximate area under the curve.
- \( n \) is the number of discrete points.
- \( x_i \) are the \( x \)-coordinates of the points.
- \( y_i \) are the \( y \)-coordinates of the points.
- The term \(\frac{(y_i + y_{i+1})}{2}\) represents the average height of the function between \(x_i\) and \(x_{i+1}\).
- The difference \((x_{i+1} - x_i)\) represents the width of the interval.

In the context of the provided code:
**Area Calculation**:
   The area is calculated as follows:

$$
\text{area} = |A| = \left| \int_{x_{\text{closed}}} y_{\text{closed}} \, dx \right| \approx \text{np.abs}\left(\text{np.trapz}(y_{\text{closed}}, x_{\text{closed}})\right)
$$


In [13]:
def Terraform_DataSet(Hysteresis_Sample: pd.DataFrame):

    Tidal_Volume = Hysteresis_Sample['TV-i'].values + Hysteresis_Sample['TV-e'].values
    Apparent_Pressure = Hysteresis_Sample["pressure"].values

    Polynomial_Area = np.abs(np.trapz(Apparent_Pressure, Tidal_Volume))

    Hull_Data = np.column_stack((Tidal_Volume, Apparent_Pressure))
    Hull = ConvexHull(Hull_Data)
    Hull_Area = Hull.volume

    ft_Tidal_Volume = np.abs(fft(Tidal_Volume))[:5]
    ft_Pressure = np.abs(fft(Apparent_Pressure))[:5]

    features = {
        "Polynomial_Area": Polynomial_Area,
        "Hull_Area": Hull_Area,
        **{f"ft_Tidal_Volume_{i+1}": coef for i, coef in enumerate(ft_Tidal_Volume)},
        **{f"ft_Pressure_{i+1}": coef for i, coef in enumerate(ft_Pressure)}
    }

    features_df = pd.DataFrame(features, index=[0])
    enhanced_Hysteresis_Sample = pd.concat([Hysteresis_Sample.reset_index(drop=True), features_df], axis=1)

    return enhanced_Hysteresis_Sample




def Feature_Engineering(DataSet: pd.DataFrame):
    DataSet = DataSet.sort_values(by=['breath_id', 'time_step']).copy()
    Terraformed_Data = []

    for breath_id, group in DataSet.groupby('breath_id'):
        Hysteresis_Sample = Terraform_DataSet(group)  
        Terraformed_Data.append(Hysteresis_Sample)

    DataSet = pd.concat(Terraformed_Data, ignore_index=True)

    return DataSet


def Clean_DataSet(DataSet: pd.DataFrame):
    DataSet.dropna(subset=['pressure', 'Polynomial_Area'], inplace=True)
    DataSet = DataSet[DataSet['Polynomial_Area'] <= 417]
    DataSet = DataSet[~((DataSet['Polynomial_Area'] > 73) & (DataSet['Polynomial_Area'] < 150))]
    DataSet.drop(DataSet.columns[[0,2,3]], axis =1, inplace=True)
    # Within These ranges The individual Breath cycles do not connect or are incomplete
    return DataSet

<img src="./HeatMap.png" alt="Scatter Matrix" style="opacity: 1;" />

# |Cleaning The DataSet

In [ ]:
DataSet = Clean_DataSet(DataSet)
DataSet.info()
DataSet.to_csv("ClusterSet_26Hz.csv")
del DataSet

<class 'pandas.core.frame.DataFrame'>
Index: 3067600 entries, 0 to 6035999
Data columns (total 6 columns):
 #   Column           Dtype  
---  ------           -----  
 0   breath_id        float64
 1   time_step        float64
 2   TV-i             float64
 3   TV-e             float64
 4   pressure         float64
 5   Polynomial_Area  float64
dtypes: float64(6)
memory usage: 163.8 MB


> # Machine Learning Starts Soon :)

> # Converging data for **Unsupervised Learning**
-  a.k.a **Theoretical Rocket Surgery**

In [ ]:
DataSet = Feature_Engineering(DataSet)

In [ ]:
TF_ClusterSet = DataSet.drop_duplicates(subset='breath_id', keep='first').reset_index(drop=True)

38345

In [26]:
#TF_ClusterSet.drop(TF_ClusterSet.columns[[1,2,3,4]], axis =1, inplace=True)
#TF_ClusterSet.head()
TF_ClusterSet.to_csv("TF_Cluster26Hz.csv")